In [1]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [4]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [12]:
def value_iteration(env, theta=0.0001, discount_factor=1.0):
    """
    Value Iteration Algorithm.
    
    Args:
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
        
    Returns:
        A tuple (policy, V) of the optimal policy and the optimal value function.        
    """

    def evaluate_once(policy, V):
        for s in range(env.nS):
            current_v = 0
            for a, action_prob in enumerate(policy[s]):
                for prob, ns, r, done in env.P[s][a]:
                    current_v += action_prob * prob * (r + discount_factor * V[ns])
            V[s] = current_v
        return V
    
    def one_step_lookahead(V, s):
        values = np.zeros(env.nA)
        for a in range(env.nA):
            for prob, ns, r, done in env.P[s][a]:
                values[a] += prob * (r + discount_factor * V[ns])
        return values


    # get optimal value function by doing one step lookahead
    V = np.zeros(env.nS)
    while True:
        delta = 0
        # V = evaluate_once(policy, V)
        for s in range(env.nS):
            best_action_v = np.max(one_step_lookahead(V, s))
            delta = np.max([delta, np.abs(best_action_v - V[s])])
            V[s] = best_action_v
        if delta < theta:
            break

    # create a deterministic policy with the optimal value function
    policy = np.zeros([env.nS, env.nA])
    for s in range(env.nS):
        values = one_step_lookahead(V, s)
        best_action = np.argmax(values)
        policy[s, best_action] = 1
    return policy, V

In [20]:
import time
start_time = time.perf_counter()
policy, v = value_iteration(env)
end_time = time.perf_counter()
print(f"Time taken: {(end_time - start_time)*1000} mseconds")
print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

Time taken: 0.6556440002896124 mseconds
Policy Probability Distribution:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



In [14]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)